In [6]:
import json
import openai
from dotenv import load_dotenv
from tqdm import tqdm
import os

# ✅ .env 파일에서 API 키 불러오기
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ 상위 타입 설명 + 하위 타입 예시
system_prompt = """
You are a deceptive pattern classifier. You must classify each input pattern into ONE of the following 7 categories:

1. social_proof
- Definition: Tricks the user into believing that many people are interested in a product or have recently purchased it.
- Examples: fake customer activity popups, fake testimonials, "X people are viewing this item now"

2. misdirection
- Definition: Confuses users by altering expected interface behavior, such as placing buttons in unusual locations, using misleading hierarchy, or emphasizing the wrong action.
- Examples: confirmshaming, trick wording, visually obscured alternatives

3. urgency
- Definition: Creates pressure by claiming time is running out to push users into making a quick decision.
- Examples: countdown timers, "sale ends in 5 minutes", "act now"

4. forced_action
- Definition: Requires users to complete an unrelated action in order to proceed, such as signing up before viewing information.
- Examples: forced login to see details, mandatory email access, bundled consent

5. obstruction
- Definition: Makes it difficult for users to cancel or opt out, usually requiring more steps than sign-up.
- Examples: hidden cancellation buttons, requiring customer service calls, unclear opt-out

6. sneaking
- Definition: Hides or adds unexpected charges or items during checkout without the user's awareness.
- Examples: hidden fees, hidden subscriptions, pre-selected upsells

7. scarcity
- Definition: Falsely claims a product is in limited supply to trigger fear of missing out.
- Examples: "only 2 left", fake stock counters, fake demand messages

Classify each pattern below into one of the above categories. Respond only with the category name, like: "sneaking" or "misdirection".
"""

# ✅ 사용자 입력용 프롬프트 생성 함수
def create_user_prompt(name, definition, example):
    return f"""
Pattern Name: {name}
Definition: {definition}
Example: {example}

Which category does this pattern belong to?
"""

# ✅ JSON 파일 읽기
with open("dark_patterns.json", "r", encoding="utf-8") as f:
    dark_patterns = json.load(f)

# ✅ 분류 결과 저장용 dict
results = {}

# ✅ 하나씩 처리
for name, item in tqdm(dark_patterns.items()):
    try:
        user_prompt = create_user_prompt(name, item["definition"], item["example"])

        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )

        category = response["choices"][0]["message"]["content"].strip().lower()
        results[name] = category
        print(f"[✓] {name} → {category}")

    except Exception as e:
        print(f"[X] {name} 처리 중 오류: {e}")
        results[name] = "error"

# ✅ 결과 저장
with open("dark_patterns_mapped.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("\n🎉 분류 완료! 결과는 'dark_patterns_mapped.json'에 저장됨.")


100%|██████████| 16/16 [00:00<00:00, 13187.04it/s]

[X] comparison_prevention 처리 중 오류: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.
[X] confirmshaming 처리 중 오류: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.
[X] Disguised_ads 처리 중 오류: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_A

In [ ]:
import json

# ✅ 1. 기존 분류 결과 불러오기
with open("dark_patterns_mapped.json", "r", encoding="utf-8") as f:
    mapped_data = json.load(f)

# ✅ 2. 상위 타입별로 하위 항목 묶기
grouped = {}
for subtype, supertype in mapped_data.items():
    if supertype not in grouped:
        grouped[supertype] = []
    grouped[supertype].append(subtype)

# ✅ 3. 결과 저장
with open("dark_patterns_mapped.json", "w", encoding="utf-8") as f:
    json.dump(grouped, f, indent=2, ensure_ascii=False)

print("✅ 상위 타입별 하위 항목 묶기 완료! 결과는 'dark_patterns_mapped.json'에 저장됨.")

In [ ]:
import json
import openai
import os
from tqdm import tqdm
from dotenv import load_dotenv

# ✅ .env 파일에서 API 키 불러오기
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# ✅ 2. 상위 타입 묶음 (grouped_by_supertype.json)
with open("dark_patterns_mapped.json", "r", encoding="utf-8") as f:
    grouped = json.load(f)

# ✅ 3. 각 패턴 설명 데이터 (dark_patterns.json)
with open("dark_patterns.json", "r", encoding="utf-8") as f:
    pattern_data = json.load(f)

# ✅ 4. 프롬프트 생성 함수 (행동 기반 predicate 유도)
def make_action_style_prompt(supertype, subtype_list, pattern_data):
    merged_text = ""
    for subtype in subtype_list:
        definition = pattern_data[subtype]["definition"]
        example = pattern_data[subtype]["example"]
        merged_text += f"\nSubtype: {subtype}\nDefinition: {definition}\nExample: {example}\n"

    prompt = f"""
You are an expert in Prolog and deceptive UI pattern detection.

You will be given multiple dark pattern subtypes that belong to the same high-level category: "{supertype}".

Your task is to generate 3 Prolog-style predicates that describe concrete, detectable UI behaviors used in these patterns.

Each predicate should:
- Use an action-based name like "hide_details", "auto_select_option", "inject_fake_timer"
- Be lowercase with underscores
- Include 1–2 arguments that represent UI elements or interface states
- Be implementation-friendly (for a detection engine)

Here are the related subtypes:\n{merged_text}

Return ONLY a JSON array of 3 predicate strings.
"""
    return prompt

# ✅ 5. GPT로 상위 타입당 3개 predicate 생성
final_results = {}

for supertype, subtype_list in tqdm(grouped.items()):
    prompt = make_action_style_prompt(supertype, subtype_list, pattern_data)

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )

        raw_output = response["choices"][0]["message"]["content"].strip()
        print(f"\n📥 GPT Response for '{supertype}':\n{raw_output}\n")

        # JSON 파싱 시도
        predicates = json.loads(raw_output)
        final_results[supertype] = predicates

    except json.JSONDecodeError as je:
        print(f"[JSON ERROR] {supertype}: {je}")
        final_results[supertype] = ["json_parse_error"]

    except Exception as e:
        print(f"[X] Error for {supertype}: {e}")
        final_results[supertype] = ["other_error"]

# ✅ 6. 결과 저장
with open("predicates_type_7.json", "w", encoding="utf-8") as f:
    json.dump(final_results, f, indent=2, ensure_ascii=False)

print("\n✅ 저장 완료: predicates_type_7.json 에 predicate 결과 저장됨.")
